<a href="https://colab.research.google.com/github/malvarezgarcia1213-png/Project-I-Deep-Learning-Image-Classification-with-CNN/blob/main/Animals_CNN_DEPLOYMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. IMPORT LIBRARIES (DEPLOYMENT NOTEBOOK)

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input
from PIL import Image
import gradio as gr
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2. LOAD MODEL AND CLASS NAMES

MODEL_PATH = '/content/drive/MyDrive/animals_project/models/efficientnet_finetuned_v3.keras'
CLASS_PATH = '/content/drive/MyDrive/animals_project/class_names.npy'

# Load the fine-tuned model (EfficientNetB0)
model = tf.keras.models.load_model(MODEL_PATH, compile=False)

# Load class order
class_names = np.load(CLASS_PATH, allow_pickle=True)
# Translate class names from Italian to English
translation_map = {
    'cane': 'dog',
    'cavallo': 'horse',
    'elefante': 'elephant',
    'farfalla': 'butterfly',
    'gallina': 'chicken',
    'gatto': 'cat',
    'mucca': 'cow',
    'pecora': 'sheep',
    'ragno': 'spider',
    'scoiattolo': 'squirrel'
}

class_names = np.array([translation_map.get(name, name) for name in class_names])

print("Model and class names loaded successfully.")
print(f"Number of classes: {len(class_names)}")


Model and class names loaded successfully.
Number of classes: 10


In [ ]:
# 3. IMAGE PREPROCESSING AND PREDICTION FUNCTION

def predict_image(image):
    # Convert to RGB and resize to 224x224
    img = image.convert('RGB').resize((224, 224))
    img_array = np.array(img).astype('float32')

    # Apply EfficientNet preprocessing (only once)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Predict probabilities
    predictions = model.predict(img_array)[0]

    # Normalize predictions to ensure valid percentages
    predictions = predictions / np.sum(predictions)

    # Get top-3 predictions
    top_indices = predictions.argsort()[-3:][::-1]
    results = {
        class_names[i]: float(predictions[i])
        for i in top_indices
    }

    return results


In [ ]:
# 4. GRADIO INTERFACE (COLAB TEST VERSION)

import gradio as gr

# Define Gradio interface
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload an animal image"),
    outputs=gr.Label(num_top_classes=3, label="Predictions"),
    title="Animals-10 Classifier (EfficientNetB0 Fine-Tuned)",
    description="Upload an animal image to get the top-3 predictions.",
    examples=[
        ["/content/drive/MyDrive/animals_project/test_images/example.jpg"]
    ]
)

# Launch the Gradio app
interface.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://47b5e19f152c7d8a80.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://47b5e19f152c7d8a80.gradio.live
